# MITx (Mid-lat ionospheric trough)

> Abstract: Access to the `MITx_LP` and `MITxTEC` products from the PRISM project. These give information about the boundaries and minima of the midlatitude ionospheric trough derived from the Langmuir Probe (LP) measurements and Total Electron Content (TEC) respectively.

See also:
- More notebooks: https://nbviewer.org/github/pacesm/jupyter_notebooks/tree/master/PRISM/
- Project page and documentation: https://earth.esa.int/eogateway/activities/plasmapause-related-boundaries-in-the-topside-ionosphere-as-derived-from-swarm-measurements

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
from viresclient import SwarmRequest

## PRISM product information

The [PRISM project](https://earth.esa.int/eogateway/activities/plasmapause-related-boundaries-in-the-topside-ionosphere-as-derived-from-swarm-measurements) (*Plasmapause Related boundaries in the topside Ionosphere as derived from Swarm Measurements*) has two data product outputs derived from Swarm which provide the minima of the mid-latitude ionospheric trough (MIT):  
`MITx_LP`: derived from the Langmuir Probe (LP)  
`MITxTEC`: derived from the GPS Total Electron Content (TEC)  

These are implemented in VirES as four collection types:  
`SW_OPER_MITx_LP_2F`  
`SW_OPER_MITx_LP_2F:ID`  
`SW_OPER_MITxTEC_2F`  
`SW_OPER_MITxTEC_2F:ID`  
where `x` = `A/B/C` for each spacecraft.

The base collections (`SW_OPER_MITx_LP_2F` and `SW_OPER_MITxTEC_2F`) contain information on the positions of the MIT minima. The sub-collections with `:ID` appended contain information on the boundaries.

Similarly, the midnight plasmapause index (PPI) is provided in the collections:  
`SW_OPER_PPIxFAC_2F`  
`SW_OPER_PPIxFAC_2F:ID`

For details about the products and their derivation, see the documentation links on the [project page](https://earth.esa.int/eogateway/activities/plasmapause-related-boundaries-in-the-topside-ionosphere-as-derived-from-swarm-measurements).

For further details about the implementation in VirES, see:
- https://nbviewer.jupyter.org/github/pacesm/jupyter_notebooks/blob/master/PRISM/PRISM_00_data_access.ipynb

Let's now look at the variables available within each collection:

In [ ]:
request = SwarmRequest()
print("MITx_LP variables:\n", request.available_measurements("SW_OPER_MITA_LP_2F"))
print("MITx_LP:ID variables:\n", request.available_measurements("SW_OPER_MITA_LP_2F:ID"))

In [ ]:
request = SwarmRequest()
print("MITxTEC variables:\n", request.available_measurements("SW_OPER_MITATEC_2F"))
print("MITxTEC:ID variables:\n", request.available_measurements("SW_OPER_MITATEC_2F:ID"))

The difference between the `LP`-derived and `TEC`-derived datasets is the presence of either the variables `Ne, Te` or `TEC` reflecting their origin (the `LP` one being derived from electron density and temperature measurements from the Langmuir Probe, and the `TEC` one being derived from the Total Electron Content estimates).

In both cases, the dataset is split between the base collection, e.g. `SW_OPER_MITA_LP_2F`, and the sub-collection marked with `:ID`, e.g. `SW_OPER_MITA_LP_2F:ID`. This separation is made in order to fit the data model requirements within VirES. The base collections contain information about the *minima* of the MIT, and the `:ID` collections contain the identified *boundaries* (e.g. poleward/equatorward edges etc) encoded within the variables, `PointType`, and `PositionQuality`.

In [ ]:
mitx_lp_vars = [
    # 2D variable: orbit number and quarter-orbit
    "Counter",
    # Redundant with VirES auxiliaries, QDLat, QDLon, MLT
    "Latitude_QD", "Longitude_QD", "MLT_QD",
    # Parameters characterising the identified MIT minima
    # McIlwain L-parameter, solar zenith angle, electron density and temperature
    "L_value", "SZA", "Ne", "Te",
    # Depth and relative change in density of the MIT
    "Depth", "DR",
    # Width of the MIT, in QD latitudes, and in L-values
    "Width", "dL",
    # Change in log electron density divided by QD latitude difference,
    # at the poleward and equatorward walls of the MIT
    "PW_Gradient", "EW_Gradient",
    # Quality indicator of the detection quality
    "Quality"
]

In [ ]:
mitx_lpid_vars = [
    # As above
    "Counter", "Latitude_QD", "Longitude_QD", "MLT_QD", "L_value", "SZA", "Ne", "Te",
    # Identifier to mark the type of the detected point (see below)
    "PointType",
    # Detection quality of each position
    "Position_Quality"
]

`PointType` is not present within the source data files, but is identified from the `*_ID` columns, and has values as follows:

| Value  | Hex. | Bin. | Meaning |
|---|---|---|---|
| 0 | 0x0 | 0b0000 | LP MIT equatorward edge of the equatorward wall |
| 1 | 0x1 | 0b0001 | LP MIT poleward edge of the equatorward wall |
| 2 | 0x2 | 0b0010 | LP MIT equatorward edge of poleward wall |
| 3 | 0x3 | 0b0011 | LP MIT poleward edge of the poleward boundary |
| 4 | 0x4 | 0b0100 | LP SETE equatorward bounding position |
| 5 | 0x5 | 0b0101 | LP SETE poleward bounding position |
| 6 | 0x6 | 0b0110 | LP Te peak position |

## MITx_LP example

In [ ]:
start = dt.datetime(2016, 1, 1)
end = dt.datetime(2016, 1, 8)

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MITA_LP_2F")
request.set_products(measurements=mitx_lp_vars, auxiliaries=["QDLat", "MLT"])
data = request.get_between(
    start_time=start, end_time=end, asynchronous=False, show_progress=False
)
ds = data.as_xarray()

Oops! I made a mistake with loading the "units" metadata (the PRISM products diverge from the conventional naming). Here's a temporary fix (to be fixed in next version of `viresclient`).

In [ ]:
with data.contents[0].open_cdf() as cdf:
    for data_var in ds.data_vars:
        units = cdf.varattsget(data_var).get("UNIT", "-")
        ds[data_var].attrs["units"] = units

In [ ]:
ds

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MITA_LP_2F:ID")
request.set_products(measurements=mitx_lpid_vars, auxiliaries=["QDLat", "MLT"])
data_id = request.get_between(
    start_time=start, end_time=end, asynchronous=False, show_progress=False
)
ds_id = data_id.as_xarray()
# temporary fix again
with data_id.contents[0].open_cdf() as cdf:
    for data_var in ds_id.data_vars:
        units = cdf.varattsget(data_var).get("UNIT", "-")
        ds_id[data_var].attrs["units"] = units

In [ ]:
ds_id

## Visualisation

Some visualisation examples will be added here soon. For now, refer to:
- https://nbviewer.org/github/pacesm/jupyter_notebooks/tree/master/PRISM/